In [5]:
import pandas as pd
import numpy as np
import scipy
# numpy version should be 1.13, because of the heaviside function
from scipy.stats import pearsonr, logistic
from scipy.special import expit
from scipy.spatial.distance import squareform
from scipy.optimize import minimize
import time
import pdb

In [6]:
def load_data():
    df = pd.read_csv('data/Daejeon_dataset.csv', delimiter='\t', index_col=False)
    return df

In [7]:
def get_pref_mats(df):
    '''
    This function generates the check-in matrix and sentiment-preference matrix
    
    Input:
    df, dataFrame. Return from load_data function

    Output:
    1. pref_checkin, check-in preference matrix
    2. pref_sentiment, sentiment preference matrix
    '''
    mem_id = sorted(df['Member ID'].unique()); loc_id = sorted(df['Restaurant ID'].unique())
    pref_checkin = pd.DataFrame(0, index=mem_id, columns=loc_id)
    pref_sentiment = pd.DataFrame(0, index=mem_id, columns=loc_id)
    
    for index, row in df.iterrows():
        # make sentiment preference matrix
        member = row['Member ID']; restaurant = row['Restaurant ID']; rating = row['Rating']
        pref_sentiment.loc[member, restaurant] = rating
        checkin = pref_checkin.loc[member, restaurant]
        if checkin == 0:
            checkin = 1
        elif checkin > 0:
            checkin += 1

        if checkin >= 3:
            checkin = 3

        pref_checkin.loc[member, restaurant] = checkin
    
    pref_checkin = np.array(pref_checkin); pref_sentiment = np.array(pref_sentiment)
    return pref_checkin, pref_sentiment

In [8]:
df = load_data()
pref_checkin, pref_sentiment = get_pref_mats(df)

In [9]:
def compute_pref_final(pref_checkin, pref_sentiment):
    '''
    This function calculates the final preference matrix. 
    Equation (1) from the paper
    
    Input:
    1. pref_checkin, check-in preference matrix
    2. pref_sentiment, sentiment preference matrix

    Output:
    pref_final, final preference matrix
    '''
    pref_final = pref_checkin - np.sign(pref_checkin - pref_sentiment) * np.heaviside(np.abs(pref_checkin - pref_sentiment)-2, 0.5)
    return pref_final

In [10]:
Z = 5
N = len(df['Member ID'].unique()); I = len(df['Restaurant ID'].unique())
pref_final = compute_pref_final(pref_checkin, pref_sentiment)
print(pref_final.shape)

(1171, 854)


In [11]:
def get_UV(N, I, Z):
    U = np.random.rand(N, Z)
    V = np.random.rand(Z, I)
    
    return U, V

In [12]:
def get_sim_u(pref_final):
    '''
    This function returns similarity of users

    Input: pref_final, which is the return matrix of the compute_pref_final function
    Output: similarity of users, N * N matrix
    '''
    N, _ = pref_final.shape
    sim_u = []
    for n in range(N):
        temp = []
        for i in range(N):
            temp.append(pearsonr(pref_final[n], pref_final[i])[0])
        sim_u.append(temp)

    sim_u = np.array(sim_u)
    return sim_u


In [13]:
def get_sim_v(df):
    '''
    For two venues, the similarity score is set to 1 if both venues have the same sub-category in Foursquare
    and set 0 if there is no overlapping sub-category

    If two restaurants have the same cuisine code, similarity score is set to 1, else 0.
    '''
    location = []
    for index, row in df.iterrows():
        tmp = [row['Restaurant ID'], row['Restaurant code']]
        if tmp not in location:
            location.append(tmp)

    sim_v = []
    I = len(location)
    
    for i in range(I):
        temp = []
        current_code = location[i][1]
        for j in range(I):
            if location[j][1] == current_code:
                temp.append(1)
            else:
                temp.append(0)
        sim_v.append(temp)
        
    sim_v = np.array(sim_v)
    return sim_v

In [14]:

def get_coefficient(pref_final, sim_u, sim_v, U, V):
    '''
    lambda_u = sigma^2_R / sigma^2_U
    lambda_v = sigma^2_R / sigma^2_V
    alpha = sigma^2_R / sigma^2_simU
    beta = sigma^2_R / sigma^2_simV
    '''
    var_R = np.var(pref_final)
    lambda_u = var_R / np.var(U)
    lambda_v = var_R / np.var(V)
    alpha = var_R / np.var(sim_u)
    beta = var_R / np.var(sim_v)
    
    return lambda_u, lambda_v, alpha, beta

In [15]:
def get_log_posterior(U, V, pref_final, sim_u, sim_v, lambda_u, lambda_v, alpha, beta, N, I, Z):
    U = scipy.resize(U, (N, Z))
    V = scipy.resize(V, (Z, I))
    first_term = np.sum(pref_final - expit(U @ V))
    second_term = lambda_u * np.sum(U @ U.T) + lambda_v * np.sum(V @ V.T)
    third_term = alpha * np.sum((U - (sim_u @ U)) @ (U - (sim_u @ U)).T)
    fourth_term = beta * np.sum((V.T - (sim_v @ V.T)) @ (V.T - (sim_v @ V.T)).T)
    log_posterior = 0.5 * (first_term + second_term + third_term + fourth_term)
#     print("log_posterior:", log_posterior)
    return log_posterior

In [16]:
start = time.time()
U, V = get_UV(N, I, Z)
sim_u = get_sim_u(pref_final)
sim_v = get_sim_v(df)
lambda_u, lambda_v, alpha, beta = get_coefficient(pref_final, sim_u, sim_v, U, V)
end = time.time() - start
print(end)
print(sim_u)

78.11264824867249
[[ 1.         -0.00117233 -0.0016589  ..., -0.00117233 -0.00117233
  -0.00117233]
 [-0.00117233  1.          0.70669218 ..., -0.00117233 -0.00117233
  -0.00117233]
 [-0.0016589   0.70669218  1.         ..., -0.0016589  -0.0016589
  -0.0016589 ]
 ..., 
 [-0.00117233 -0.00117233 -0.0016589  ...,  1.         -0.00117233
  -0.00117233]
 [-0.00117233 -0.00117233 -0.0016589  ..., -0.00117233  1.         -0.00117233]
 [-0.00117233 -0.00117233 -0.0016589  ..., -0.00117233 -0.00117233  1.        ]]


In [18]:
log_posterior = get_log_posterior(U, V, pref_final, sim_u, sim_v, lambda_u, lambda_v, alpha, beta, N, I, Z)
print(log_posterior)

958154307.692


In [19]:
def get_grad_u(U, V, pref_final, sim_u, sim_v, lambda_u, lambda_v, alpha, beta, N, I, Z):
    '''
    A: U, B: V 
    '''
    U = scipy.resize(U, (N, Z))
    V = scipy.resize(V, (Z, I))
#     U = U.reshape(1171, 5)

#     pdb.set_trace()
    grad_u_first = (logistic.pdf(U @ V) * (expit(U @ V) - pref_final)) @ V.T
    grad_u_second = lambda_u * U + alpha * (U - sim_u @ U)
    grad_u_third = -alpha * (sim_u @ (U - sim_u @ U))
    grad_u = grad_u_first + grad_u_second + grad_u_third
    grad_u = np.ndarray.flatten(grad_u)
#     print("gradient_u:", grad_u)
    return grad_u

def get_grad_v(U, V, pref_final, sim_u, sim_v, lambda_u, lambda_v, alpha, beta, N, I, Z):
#     U = U.reshape(1171, 5)

    U = scipy.resize(U, (N, Z))
    V = scipy.resize(V, (Z, I))
    grad_v_first = (logistic.pdf(U @ V) * (expit(U @ V)-pref_final)).T @ U
    grad_v_second = (lambda_v * V).T + beta * (V.T - sim_v @ V.T)
    grad_v_third = -beta * (sim_v @ (V.T - sim_v @ V.T))
    grad_v = grad_v_first + grad_v_second + grad_v_third
    grad_v = np.ndarray.flatten(grad_v)
#     print("gradient_v:", grad_v)
    return grad_v

In [20]:
cnt = 0
while True:
    cnt += 1
    print("---------------------------------------------------------------")
    print("%d th iteration" % cnt)
    print("---------------------------------------------------------------")
    u_res = minimize(get_log_posterior,
                     x0 = U, args = (V, pref_final, sim_u, sim_v, lambda_u, lambda_v, alpha, beta, N, I, Z),
                     jac = get_grad_u)

    v_res = minimize(get_log_posterior,
                     x0 = V, args = (U, pref_final, sim_u, sim_v, lambda_u, lambda_v, alpha, beta, N, I, Z),
                     jac = get_grad_v)

    estimated_U = u_res.x.reshape(N, Z)
    estimated_V = v_res.x.reshape(Z, I)
    
    cond = np.sqrt(np.sum(np.square(U - estimated_U)) + np.sum(np.square(V - estimated_V)))
    print("condition value:", cond)
    print("U:", U)
    print("V:", V)
    print("estimated_U:", estimated_U)
    print("estimated_V:", estimated_V)
    condition = cond < 1e-01
    print("condition:", condition)

    if condition:
        break

    U, V = estimated_U, estimated_V

---------------------------------------------------------------
1 th iteration
---------------------------------------------------------------


/Users/Lumin/anaconda3/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:3060: RuntimeWarning: overflow encountered in exp
  return -x - 2. * sc.log1p(np.exp(-x))


condition value: 85.3114288879
U: [[ 0.26787989  0.0497855   0.35808054  0.70019963  0.1449181 ]
 [ 0.52997257  0.08187515  0.90931762  0.51978989  0.08713174]
 [ 0.95832126  0.58291143  0.42874396  0.73744857  0.9306805 ]
 ..., 
 [ 0.66144285  0.69538433  0.11971926  0.01538377  0.31769275]
 [ 0.60878521  0.43905141  0.31765034  0.5123209   0.26501237]
 [ 0.01332782  0.16929433  0.121651    0.9910458   0.72520759]]
V: [[ 0.29174646  0.09987541  0.65215621 ...,  0.55594912  0.57337583
   0.6126712 ]
 [ 0.14396818  0.93760949  0.61922026 ...,  0.71252439  0.89533332
   0.6796808 ]
 [ 0.55934627  0.86656676  0.87829258 ...,  0.11597187  0.55112714
   0.9941208 ]
 [ 0.08974699  0.16219807  0.45280324 ...,  0.78923997  0.54680038
   0.55744621]
 [ 0.21954665  0.70219724  0.76997909 ...,  0.86280184  0.47324486
   0.77029299]]
estimated_U: [[-1.10161026 -1.18378978 -1.10386407 -0.67314731 -1.19204213]
 [ 0.60013617  0.15029873  0.9100951   0.55898898  0.18771146]
 [ 0.65639714  0.23128143  

In [37]:
MAE, RMSE = compute_metrics(U, V, pref_final)
print(MAE)
print(RMSE)

1.4786886481
1.662952847


In [36]:
def compute_metrics(U, V, pref_final):
    R_hat = U @ V
    T = pref_final.shape[0] * pref_final.shape[1]
    MAE = np.sum(np.abs(pref_final - R_hat)) / T
    RMSE = np.sqrt(np.sum(np.square(pref_final - R_hat)) / T)
    return MAE, RMSE

In [52]:
pref_final
test = np.zeros(pref_final.shape)
train = pref_final.copy()
for user in range(pref_final.shape[0]):
    visited = pref_final[user, :].nonzero()[0]
    if len(visited) == 1:
        pass
    elif len(visited) > 3:
        test_pref = np.random.choice(pref_final[user, :].nonzero()[0], size=3, replace=False)
        train[user, test_pref] = 0.
        test[user, test_pref] = pref_final[user, test_pref]
    
assert(np.all((train * test) == 0))


In [66]:
print(sim_v.shape)
print(np.amax(sim_u[0][1:]))
print(np.argmax(sim_u[0][1:]))
print(sim_u[0][17])

(854, 854)
1.0
16
1.0
